# Régression logistique multinomiale

Nous avons déjà vu la régression logistique binomiale où la variable dépendante est de nature catégorielle avec 2 classes possibles.

Avec une **régression logistique multinomiale**, nous pouvons aussi traiter le cas d'une variable catégorielle avec n'importe quel nombre de classes.

Si notre variable dépendante est de nature ordinale (c'est-à-dire des catégories avec un ordre bien établi), nous allons opter pour une **régression logistique ordinale**.

Pour calculer une régression logistique multinomiale dans R, nous avons plusieurs choix de fonctions à disposition, dans des libraires différentes. En raison de sa relative simplicité, nous allons utiliser ici la libraire [__nnet__](https://cran.r-project.org/web/packages/nnet) et plus spécifiquement la fonction __multinom__.

Nous pouvons charger la libraire avec

In [1]:
library(nnet)

et si ça donne une erreur disant que la libraire n'est pas disponible, nous l'installons d'abord avec:

In [ ]:
install.packages(c("nnet"))

Comme exemple concret, nous reprenons le même exemple que celui pour la régression logistique, mais cette fois en utilisant les 3 catégories pour la variable dépendante. Pour rappel, nous essayons de prédire la typologie de la région selon l'une des 3 catégories suivante:

- Centres urbains
- Espaces sous influence des centres urbains
- Espaces hors influence des centres urbains

Pour des raisons pratiques, nous avons ajouté une colonne avec un libellé plus court, à savoir "CENTRE", "AGGLO" et "RURAL".

Les variable indépendantes sont:

- **NADHO**: proportion de femmes et hommes de ménage
- **NADRET**: proportion de personnes actives à la retraite
- **AD3PRIM**: proportion de personnes actives dans le secteur primaire
- **AD3SEC**: proportion de personnes actives dans le secteur secondaire

Il faut encore noter que nos variables dépendantes doivent avoir grosso modo des valeurs entre 0 et 1. Étant donné que nous avons des proportions, ceci est déjà donné. Dans d'autres cas, il faut faire un petit calcul de transformation.

Comme d'habitude, nous commençons par la lecture du tableau de données:

In [9]:
d = read.csv(file="ch-socioeco-typologie.tsv", sep="\t")

Nous regardons de plus près la variable dépendante:

In [10]:
summary(d$typologie_ofs_court)

AGGLO CENTRE  RURAL 
   291    849    225

À partir du jeux de données initial, nous créons un jeu de données de calibrage (_training data_), et un jeu de données de test pour la validation. Le jeux de données de calibrage contient un échantillon aléatoire avec le 90% des données.

In [18]:
idx = sample(nrow(d), size=0.9*nrow(d))
dtrain = d[idx,]
dtest = d[-idx,]

Nous pouvons maintenant calculer la régression logistique multinomiale:

In [19]:
regmlogit = multinom(typologie_ofs_court ~ NADHO + NADRET + AD3PRIM + AD3SEC, data=dtrain)

# weights:  18 (10 variable)
initial  value 1349.095890 
iter  10 value 698.883679
iter  20 value 682.951382
iter  30 value 682.800403
final  value 682.799559 
converged


et puis afficher l'information sur les coefficients, la déviance et l'AIC, comme déjà pour la régression logistique binomiale:

In [20]:
summary(regmlogit)

Call:
multinom(formula = typologie_ofs_court ~ NADHO + NADRET + AD3PRIM + 
    AD3SEC, data = dtrain)

Coefficients:
       (Intercept)      NADHO   NADRET    AD3PRIM   AD3SEC
CENTRE    3.103063 -20.055914 5.050562 -108.17131 1.323254
RURAL    -5.706587  -1.494709 6.699037   26.19042 8.884455

Std. Errors:
       (Intercept)    NADHO   NADRET  AD3PRIM   AD3SEC
CENTRE   0.5278389 3.912453 1.287680 7.456027 1.279616
RURAL    0.7381347 4.577601 1.501593 4.353030 1.577600

Residual Deviance: 1365.599 
AIC: 1385.599 

Nous avons en fait 2 modèles différents, l'un pour chacune des catégories, sauf pour une catégorie. Ceci est dû au fait que nous estimons des probabilités pour chacune des catégories, et que la somme des probabilités doit obligatoirement donner 1. En conséquence, une fois que nous avons calculé la probabilité pour deux catégories, nous pouvons calculer celle de la dernière catégories avec un simple calcul.

Pour l'évaluation, nous pouvons faire un tableau comparatif simple entre les catégories prédites et les catégories effectives pour le jeu de données de calibrage et celui de test:

In [25]:
table(predict(regmlogit, newdata=dtrain), dtrain$typologie_ofs_court)

        
         AGGLO CENTRE RURAL
  AGGLO    112     29    51
  CENTRE   113    724    38
  RURAL     40     13   108

In [23]:
table(predict(regmlogit, newdata=dtest), dtest$typologie_ofs_court)

        
         AGGLO CENTRE RURAL
  AGGLO      6      3     6
  CENTRE    18     77     9
  RURAL      2      3    13

Et nous pouvons calculer la proportion de régions classées correctement, pour les deux jeux de données:

In [26]:
accuracy_train = sum(predict(regmlogit, newdata=dtrain) == dtrain$typologie_ofs_court) / nrow(dtrain)
accuracy_test = sum(predict(regmlogit, newdata=dtest) == dtest$typologie_ofs_court) / nrow(dtest)

In [27]:
accuracy_train
accuracy_test

[1] 0.7687296

[1] 0.7007299

Nous avons donc environ 77% de réponses justes pour les données de calibration, et 70% pour la validation. C'est ce dernier chiffre qui représente au mieux la qualité du modèle. Notre modèle est donc au mieux médiocre...

## NNet ??

Pour notre régression logistique multinomiale, nous avons utilisé la libraire **[nnet](https://cran.r-project.org/web/packages/nnet)**. Il s'agit d'une libraire pour les réseau de neurones artificiels (Artificial Neural Network ANN). Donc comment ça se fait qu'on utilise une libraire pour les réseaux de neurones si on veut simplement faire une régression linéaire??

En fait, on peut considérer la régression logistique comme étant un cas spécial simple d'un réseau de neurones... Et la fonction que nosu avons utilisé, `multinom` construit automatiquement un réseau de neurones en interne. **Nous avons donc bel et bien utilisé un réseau de neurones...**